In [15]:
# using Pkg
# Pkg.add("JuMP")
# Pkg.add("Gurobi")
# Pkg.add("Distances")
# Pkg.add("Distributions")
# Pkg.add("DataFrames")

In [16]:
using JuMP, Gurobi, Graphs, Plots, StatsPlots, DataFrames, Random, Printf, LinearAlgebra, Distributions, Distances, Suppressor, CSV #, PlotlyJS,
const GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18


Gurobi.Env(Ptr{Nothing} @0x000000047bf08a00, false, 0)

# Generate Data

## Constants

In [17]:
struct Warehouse #supply
    num_loc::Int
    time_horizon::Int
    distance_to_arrival::Vector{Float64}
    distance_to_departure::Vector{Float64}
    capacity::Vector{Float64}
end

struct Shipment #demand
    num_shipment::Int
    arrival_time::Vector
    departure_time::Vector
    shipment_size::Vector 
end

function ShipmentData(time_horizon, num_shipment, max_shipment_size)
    rand_shipment_time = sort(rand(1:time_horizon, (num_shipment, 2)), dims = 2)
    rand_sizes = rand(1:max_shipment_size, num_shipment)

    return Shipment(num_shipment, rand_shipment_time[:, 1], rand_shipment_time[:, 2], rand_sizes)
end

function WarehouseData(grid_size, time_horizon, warehouse_row, warehouse_col, max_shipment_size, cap_tightness)
    arrival_location = (0, 0)
    departure_location = (warehouse_row, 0)
    
    location = [(i*grid_size,j*grid_size) for i =1:warehouse_row, j = 1:warehouse_col]
    num_loc = warehouse_row * warehouse_col
    distance_to_arrival = [norm(arrival_location .- location[l]) for l in 1:num_loc]
    distance_to_departure = [norm(arrival_location .- location[l]) for l in 1:num_loc]
    #cap_tightness \in [0,1]: 1 is loose tight (uniform), 0 is null size capacity (previous it was reverse - but re-reversed!)
    capacity = rand(floor(max_shipment_size * (1-cap_tightness)):max_shipment_size, warehouse_row * warehouse_col) 
    return Warehouse(num_loc, time_horizon, distance_to_arrival, distance_to_departure, capacity)
end

WarehouseData (generic function with 1 method)

# Build Model

demand packs one supply (occupancy) and size of demand packs supply size (size) 

\begin{align}
\sum_{i \in 1:I} x_{ilt} \leq 1 \quad \text{occupancy (shipment1, shipment2, location)}\\
\sum_{i \in 1:I} d_i x_i \leq s \quad (\text{vs} d_i x_i \leq s \forall i )\quad\text{size (shipment1, location)}\\ 

\\
\sum_{i \in 1:I} x_{ij} \leq 1 \forall j \quad\text{occupancy}\\
\sum_{i \in 1:I} d_i x_{ij} \leq s_j \forall j \quad\text{size}\\
\end{align}

In [18]:
function three_slt_extended_time_space(
    warehouse_data::Warehouse,
    shipment_data::Shipment
)
    L = warehouse_data.num_loc
    S = shipment_data.num_shipment
    T = warehouse_data.time_horizon

    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 60);

    @variable(model, r[1:S,1:L,1:T] >= 0, Bin);
    @variable(model, z[1:S,1:L] >= 0, Bin);

    # supply side: 
    # can't store more than one shipment at each location and time
    @constraint(model, occupancy_pack[l in 1:L, t in 1:T],
        sum(r[s,l,t] for s in 1:S) <=1)
    
    # Each location can only hold items up to max size of the location
    @constraint(model, size_pack[l in 1:L, t in 1:T],
        sum(r[s,l,t] * shipment_data.shipment_size[s] for s in 1:S) <= warehouse_data.capacity[l] ## TODO sum over shipment or not
    );

    # Force z[s,l] to capture the location we're storing s in
    @constraint(model, no_bumping[s in 1:S, l in 1:L, t in shipment_data.arrival_time[s]:shipment_data.departure_time[s]], 
        r[s,l,t] == z[s,l])

    # Each shipment must be assigned to exactly one location during its time window:
    @constraint(model, demand_pack_supply_once[s in 1:S, t in shipment_data.arrival_time[s]:shipment_data.departure_time[s]],
        sum(r[s,l,t] for l in 1:L) == 1)

    # Each shipment must be assigned to exactly zero locations outside its time window:
    @constraint(model, mute_before_timewindow[s in 1:S, t in 1:shipment_data.arrival_time[s]-1],
        sum(r[s,l,t] for l in 1:L) == 0)

    @constraint(model, mute_after_timewindow[s in 1:S, t in shipment_data.departure_time[s]+1:T],
        sum(r[s,l,t] for l in 1:L) == 0)

    @objective(model, Min, 
        sum((warehouse_data.distance_to_arrival[l] + warehouse_data.distance_to_departure[l]) / (shipment_data.departure_time[s] - shipment_data.arrival_time[s] + 1) * r[s,l,t] 
        for s in 1:S, l in 1:L,t in 1:T));

    return model, r, z
end


three_slt_extended_time_space (generic function with 1 method)

In [19]:
function two_sl_so(
    warehouse_data::Warehouse,
    shipment_data::Shipment
)
    L = warehouse_data.num_loc
    S = shipment_data.num_shipment
    T = warehouse_data.time_horizon

    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 60);
    @variable(model, z[1:S,1:L] >= 0, Bin);

    @objective(model, Min, 
        sum((warehouse_data.distance_to_arrival[l] + warehouse_data.distance_to_departure[l]) * z[s,l] for s in 1:S, l in 1:L))


    # Each product must be assigned to exactly one location:
    @constraint(model, location_assignment[s in 1:S],
        sum(
            z[s,l]
            for l in 1:L
        ) == 1
    );

    # Each location can only hold items up to max size of the location
    @constraint(model, max_size[l in 1:L, s in 1:S],
        z[s,l] * shipment_data.shipment_size[s] <= warehouse_data.capacity[l]
    );

    # Each location can hold at most one product at any specific time:
    overlap_dict = Dict{Int64,Vector{Int64}}()
    for s1 in 1:S
        s = []
        for s2 in s1+1:S
            if shipment_data.arrival_time[s1] <= shipment_data.departure_time[s2] && shipment_data.arrival_time[s2] <= shipment_data.departure_time[s1]
                append!(s, s2)
            end
        end
        overlap_dict[s1] = s
    end

    # # Each location can hold at most one product at any specific time:
    # function find_overlap(s1)
    #     s = []
    #     for s2 in s1+1:S
    #         if s1 == s2
    #             continue
    #         end

    #         if shipment_data.arrival_time[s1] <= shipment_data.departure_time[s2] && shipment_data.arrival_time[s2] <= shipment_data.departure_time[s1]
    #             append!(s, s2)
    #         end
    #     end
    #     return s
    # end

    # @constraint(model, no_overlap[s1 in 1:S, s2 in find_overlap(s1), l in 1:L],
    #     z[s1,l] + z[s2,l] <= 1
    # );
    @constraint(model, no_overlap[s1 in 1:S, s2 in overlap_dict[s1], l in 1:L],
        z[s1,l] + z[s2,l] <= 1
    );

    return model, z
end


two_sl_so (generic function with 1 method)

In [20]:
function two_sl_cso(
    warehouse_data::Warehouse,
    shipment_data::Shipment
)
    L = warehouse_data.num_loc
    S = shipment_data.num_shipment
    T = warehouse_data.time_horizon
    
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 60);

    @variable(model, z[1:S,1:L] >= 0, Bin);

    @variable(
        model,
        1 <= r[1:S] <= L, Int
    ) ;

    @variable(
        model,
        y[1:S, 1:S], Bin
    ) ;

    @objective(model, Min, 
        sum((warehouse_data.distance_to_arrival[l] + warehouse_data.distance_to_departure[l]) * z[s,l] for s in 1:S, l in 1:L))

    # Make r[s] equal to the index of the location we're storing s in
    @constraint(model, assign_location_index[s in 1:S],
        sum(
            l * z[s,l]
            for l in 1:L
        ) == r[s]
    );

    # Each product must be assigned to exactly one location:
    @constraint(model, location_assignment[s in 1:S],
        sum(
            z[s,l]
            for l in 1:L
        ) == 1
    );

    # Each location can only hold items up to max size of the location
    @constraint(model, max_size[l in 1:L, s in 1:S],
        z[s,l] * shipment_data.shipment_size[s] <= warehouse_data.capacity[l] ## TODO sum over shipment or not
    );

    # Each location can hold at most one product at any specific time:
    overlap_dict = Dict{Int64,Vector{Int64}}()
    for s1 in 1:S
        s = []
        for s2 in s1+1:S
            if shipment_data.arrival_time[s1] <= shipment_data.departure_time[s2] && shipment_data.arrival_time[s2] <= shipment_data.departure_time[s1]
                append!(s, s2)
            end
        end
        overlap_dict[s1] = s
    end

    # function find_overlap(s1)
    #     s = []
    #     for s2 in s1:S
    #         if shipment_data.arrival_time[s1] <= shipment_data.departure_time[s2] && shipment_data.arrival_time[s2] <= shipment_data.departure_time[s1]
    #             append!(s, s2)
    #         end
    #     end
    #     return s
    # end

    # @constraint(model, no_overlap_1[s1 in 1:S, s2 in find_overlap(s1)],
    #     r[s1] - r[s2] <= -.01 + (L+5) * y[s1,s2]
    # );
    # @constraint(model, no_overlap_2[s1 in 1:S, s2 in find_overlap(s1)],
    #     r[s1] - r[s2] >= .01 - (1-y[s1,s2]) * (L+5)
    # );
    @constraint(model, no_overlap_1[s1 in 1:S, s2 in overlap_dict[s1]],
        r[s1] - r[s2] <= -.01 + (L+5) * y[s1,s2]
    );
    @constraint(model, no_overlap_2[s1 in 1:S, s2 in overlap_dict[s1]],
        r[s1] - r[s2] >= .01 - (1-y[s1,s2]) * (L+5)
    );

    return model, z
end


two_sl_cso (generic function with 1 method)

In [21]:
Random.seed!(3000)

plotdata = []
# packing S paths in LT space
L_range = [(20, 20), (30, 30)] #, (30, 75)]
S_range = [10, 100, 300] #, 100, 250]
T_range = [20, 30, 100]
cap_tightness_range = [.1, 1]
for ((r, c), s, t, cap_tightness) in Iterators.product(
    L_range,
    S_range,
    T_range,
    cap_tightness_range
)
    G = 2
    R = r
    C = c
    L = R*C
    T = t
    max_shipment_size = 10
    S = s
    #function WarehouseData(grid_size, warehouse_row, warehouse_col, max_shipment_size, time_horizon, cap_tightness)
    warehouse_data = WarehouseData(G, T, R, C, max_shipment_size, cap_tightness)
    shipment_data = ShipmentData(T, S, max_shipment_size)

    @printf "Starting 1\n"
    buildtime1 = @elapsed model1, _, _ = three_slt_extended_time_space(warehouse_data, shipment_data) #
    # optimize!(model1)
    solvetime1 = @elapsed @suppress optimize!(model1)
    objective1 = typemax(Int32)
    try
        objective1 = objective_value(model1)
    catch
    end

    @printf "Starting 2\n"
    buildtime2 = @elapsed model2, _ = two_sl_so(warehouse_data, shipment_data)#
    # optimize!(model2)
    solvetime2 = @elapsed @suppress optimize!(model2)
    objective2 = typemax(Int32)
    try
        objective2 = objective_value(model2)
    catch
    end
    

    push!(plotdata, (
        L = L,
        S = S,
        T = T,
        cap_tightness = cap_tightness,
        buildtime1 = buildtime1, 
        solvetime1 = solvetime1, 
        objective1 = objective1,
        buildtime2 = buildtime2, 
        solvetime2 = solvetime2, 
        objective2 = objective2,

    ))
end
df = DataFrame(plotdata)

@show df

Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


...

Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


df = 36×10 DataFrame
 Row │ L      S      T      cap_tightness  buildtime1  solvetime1  objective1     buildtime2   solvetime2   objective2
     │ Int64  Int64  Int64  Float64        Float64     Float64     Float64        Float64      Float64      Float64
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │   400     10     20            0.1    0.883548    0.264797     89.3596       0.270399     0.0574515     89.3596
   2 │   900     10     20            0.1    2.55467     0.606599    100.315        0.0482839    0.117666     100.315
   3 │   400    100     20            0.1    1.23822     4.67916    1979.04         4.53246      5.41804     1979.04
   4 │   900    100     20            0.1    4.35539     9.77591    1917.82         7.24719     22.1681      1917.82
   5 │   400    300     20            0.1    5.11191    12.5828     9314.33        30.5363      89.7277     19146.3
   6 │   900    300     20           

Row,L,S,T,cap_tightness,buildtime1,solvetime1,objective1,buildtime2,solvetime2,objective2
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,400,10,20,0.1,0.883548,0.264797,89.3596,0.270399,0.0574515,89.3596
2,900,10,20,0.1,2.55467,0.606599,100.315,0.0482839,0.117666,100.315
3,400,100,20,0.1,1.23822,4.67916,1979.04,4.53246,5.41804,1979.04
4,900,100,20,0.1,4.35539,9.77591,1917.82,7.24719,22.1681,1917.82
5,400,300,20,0.1,5.11191,12.5828,9314.33,30.5363,89.7277,19146.3
6,900,300,20,0.1,20.4086,27.0847,9138.17,168.795,151.786,28607.1
7,400,10,30,0.1,0.427624,0.416238,98.1249,0.0446169,0.0616525,98.1249
8,900,10,30,0.1,0.79797,0.975861,89.3596,0.0832507,0.122163,89.3596
9,400,100,30,0.1,17.2176,4.58799,1928.24,2.16845,5.848,1928.24


In [22]:
@show combine(groupby(df, [:T, :S]), [:solvetime1, :solvetime2].=> mean)
@show combine(groupby(df, [:cap_tightness]), [:solvetime1, :solvetime2].=> mean);

combine(groupby(df, [:T, :S]), [:solvetime1, :solvetime2] .=> mean) = 9×4 DataFrame
 Row │ T      S      solvetime1_mean  solvetime2_mean
     │ Int64  Int64  Float64          Float64
─────┼────────────────────────────────────────────────
   1 │    20     10         0.503894        0.0874558
   2 │    20    100         6.36601        10.2073
   3 │    20    300        18.699         120.264
   4 │    30     10         0.742053        0.0942323
   5 │    30    100         8.81185         8.73367
   6 │    30    300        29.6663        127.653
   7 │   100     10         2.12439         0.149552
   8 │   100    100        34.052           9.99992
   9 │   100    300       122.299         143.041
combine(groupby(df, [:cap_tightness]), [:solvetime1, :solvetime2] .=> mean) = 2×3 DataFrame
 Row │ cap_tightness  solvetime1_mean  solvetime2_mean
     │ Float64        Float64          Float64
─────┼─────────────────────────────────────────────────
   1 │           0.1          26.592         

In [6]:
Random.seed!(3000)

plotdata = []
# packing S paths in LT space
L_range = [(50, 50)] #, (30, 75)]
S_range = [30]#[30, 100, 200]
T_range = [10] #[10, 50, 100]
cap_tightness_range = [0.5, 1]
for ((r, c), s, t, cap_tightness) in Iterators.product(
    L_range,
    S_range,
    T_range,
    cap_tightness_range
)
    G = 2
    R = r
    C = c
    L = R*C
    T = t
    max_shipment_size = 10
    S = s
    #function WarehouseData(grid_size, warehouse_row, warehouse_col, max_shipment_size, time_horizon, cap_tightness)
    warehouse_data = WarehouseData(G, T, R, C, max_shipment_size, cap_tightness)
    shipment_data = ShipmentData(T, S, max_shipment_size)

    @printf "Starting 1\n"
    buildtime1 = @elapsed model1, _, _ = three_slt_extended_time_space(warehouse_data, shipment_data) #
    # optimize!(model1)
    solvetime1 = @elapsed @suppress optimize!(model1)
    objective1 = typemax(Int32)
    try
        objective1 = objective_value(model1)
    catch
    end

    @printf "Starting 2\n"
    buildtime2 = @elapsed model2, _ = two_sl_so(warehouse_data, shipment_data)#
    # optimize!(model2)
    solvetime2 = @elapsed @suppress optimize!(model2)
    objective2 = typemax(Int32)
    try
        objective2 = objective_value(model2)
    catch
    end
    
    @printf "Starting 3\n"
    buildtime3 = @elapsed model3, _ = two_sl_cso(warehouse_data, shipment_data)
    # optimize!(model3)
    solvetime3 = @elapsed @suppress optimize!(model3)
    objective3 = typemax(Int32)
    try
        objective3 = objective_value(model3)
    catch
    end

    push!(plotdata, (
        L = L,
        S = S,
        T = T,
        cap_tightness = cap_tightness,
        buildtime1 = buildtime1, 
        solvetime1 = solvetime1, 
        objective1 = objective1,
        buildtime2 = buildtime2, 
        solvetime2 = solvetime2, 
        objective2 = objective2,
        buildtime3 = buildtime3, 
        solvetime3 = solvetime3,
        objective3 = objective3,

    ))
end
df_l2500 = DataFrame(plotdata)

@show combine(groupby(df_l2500, [:T, :S]), [:solvetime1, :solvetime2].=> mean)
@show combine(groupby(df_l2500, [:cap_tightness]), [:solvetime1, :solvetime2].=> mean);

Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 3
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 3
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


combine(groupby(df_l2500, [:T, :S]), [:solvetime1, :solvetime2] .=> mean) = 1×4 DataFrame
 Row │ T      S      solvetime1_mean  solvetime2_mean
     │ Int64  Int64  Float64          Float64
─────┼────────────────────────────────────────────────
   1 │    10     30          4.22692           3.1088


combine(groupby(df_l2500, [:cap_tightness]), [:solvetime1, :solvetime2] .=> mean) = 2×3 DataFrame
 Row │ cap_tightness  solvetime1_mean  solvetime2_mean
     │ Float64        Float64          Float64
─────┼─────────────────────────────────────────────────
   1 │           0.5          5.12253          2.91358
   2 │           1.0          3.33131          3.30401


In [7]:
df_l2500

Row,L,S,T,cap_tightness,buildtime1,solvetime1,objective1,buildtime2,solvetime2,objective2,buildtime3,solvetime3,objective3
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2500,30,10,0.5,5.50148,5.12253,401.993,2.07887,2.91358,401.993,1.05394,60.3568,401.993
2,2500,30,10,1.0,2.16724,3.33131,477.754,1.3556,3.30401,477.754,0.0985846,60.5859,490.543


In [8]:
Random.seed!(3000)

plotdata = []
# packing S paths in LT space
L_range = [(10, 10), (20, 20), (30, 30)] #, (30, 75)]
S_range = [100]
T_range = [50]
cap_tightness_range = [0.5, 1]
for ((r, c), s, t, cap_tightness) in Iterators.product(
    L_range,
    S_range,
    T_range,
    cap_tightness_range
)
    G = 2
    R = r
    C = c
    L = R*C
    T = t
    max_shipment_size = 10
    S = s
    #function WarehouseData(grid_size, warehouse_row, warehouse_col, max_shipment_size, time_horizon, cap_tightness)
    warehouse_data = WarehouseData(G, T, R, C, max_shipment_size, cap_tightness)
    shipment_data = ShipmentData(T, S, max_shipment_size)

    @printf "Starting 1\n"
    buildtime1 = @elapsed model1, _, _ = three_slt_extended_time_space(warehouse_data, shipment_data) #
    # optimize!(model1)
    solvetime1 = @elapsed @suppress optimize!(model1)
    objective1 = typemax(Int32)
    try
        objective1 = objective_value(model1)
    catch
    end

    @printf "Starting 2\n"
    buildtime2 = @elapsed model2, _ = two_sl_so(warehouse_data, shipment_data)#
    # optimize!(model2)
    solvetime2 = @elapsed @suppress optimize!(model2)
    objective2 = typemax(Int32)
    try
        objective2 = objective_value(model2)
    catch
    end
    

    push!(plotdata, (
        L = L,
        S = S,
        T = T,
        cap_tightness = cap_tightness,
        buildtime1 = buildtime1, 
        solvetime1 = solvetime1, 
        objective1 = objective1,
        buildtime2 = buildtime2, 
        solvetime2 = solvetime2, 
        objective2 = objective2,

    ))
end
df_l_149 = DataFrame(plotdata)

Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


...

Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


UndefVarError: UndefVarError: df2 not defined

In [9]:
@show combine(groupby(df_l_149, [:T, :S]), [:solvetime1, :solvetime2].=> mean)
@show combine(groupby(df_l_149, [:cap_tightness]), [:solvetime1, :solvetime2].=> mean);

combine(groupby(df_l_149, [:T, :S]), [:solvetime1, :solvetime2] .=> mean) = 1×4 DataFrame
 Row │ T      S      solvetime1_mean  solvetime2_mean
     │ Int64  Int64  Float64          Float64
─────┼────────────────────────────────────────────────
   1 │    50    100          9.37563          6.03148
combine(groupby(df_l_149, [:cap_tightness]), [:solvetime1, :solvetime2] .=> mean) = 2×3 DataFrame
 Row │ cap_tightness  solvetime1_mean  solvetime2_mean
     │ Float64        Float64          Float64
─────┼─────────────────────────────────────────────────
   1 │           0.5         10.0288           6.80013
   2 │           1.0          8.72246          5.26284


In [98]:

CSV.write("lst2.csv", df3)


"lst2.csv"

In [96]:
df2

Row,L,S,T,cap_tightness,buildtime1,solvetime1,objective1,buildtime2,solvetime2,objective2
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,400,30,10,0,0.206036,0.507014,379.692,0.205769,0.339468,379.692
2,400,100,10,0,1.05795,1.4089,1981.06,2.84948,8.49821,1981.06
3,400,200,10,0,1.73826,3.11368,5493.57,13.0571,18.5378,5493.57
4,400,30,50,0,4.40339,1.91571,418.703,0.205869,0.349075,418.703
5,400,100,50,0,3.50574,8.29585,1936.26,5.6396,11.4683,1936.26
6,400,200,50,0,8.6485,18.9859,4942.76,12.865,69.6494,5036.77
7,400,30,100,0,1.95565,3.61797,352.774,0.186615,0.299297,352.774
8,400,100,100,0,11.1902,18.6552,1783.61,2.2423,8.80467,1783.61
9,400,200,100,0,16.827,41.495,4667.13,13.2471,68.6979,4722.24


In [ ]:
a = combine(groupby(df2, [:T, :S]), [:solvetime1, :solvetime2].=> mean)

# ctg = repeat(["Category 1", "Category 2"], inner = 5)
nam = repeat("S" .* string.(1:3), outer = 3)

In [97]:
Random.seed!(3000)

plotdata = []
# packing S paths in LT space
L_range = [(20, 20), (30, 30)] #, (30, 75)]
S_range = [30, 100]
T_range = [10, 100]
cap_tightness_range = [0.5, 1]
for ((r, c), s, t, cap_tightness) in Iterators.product(
    L_range,
    S_range,
    T_range,
    cap_tightness_range
)
    G = 2
    R = r
    C = c
    L = R*C
    T = t
    max_shipment_size = 10
    S = s
    #function WarehouseData(grid_size, warehouse_row, warehouse_col, max_shipment_size, time_horizon, cap_tightness)
    warehouse_data = WarehouseData(G, T, R, C, max_shipment_size, cap_tightness)
    shipment_data = ShipmentData(T, S, max_shipment_size)

    @printf "Starting 1\n"
    buildtime1 = @elapsed model1, _, _ = three_slt_extended_time_space(warehouse_data, shipment_data) #
    # optimize!(model1)
    solvetime1 = @elapsed @suppress optimize!(model1)
    objective1 = typemax(Int32)
    try
        objective1 = objective_value(model1)
    catch
    end

    @printf "Starting 2\n"
    buildtime2 = @elapsed model2, _ = two_sl_so(warehouse_data, shipment_data)#
    # optimize!(model2)
    solvetime2 = @elapsed @suppress optimize!(model2)
    objective2 = typemax(Int32)
    try
        objective2 = objective_value(model2)
    catch
    end
    

    push!(plotdata, (
        L = L,
        S = S,
        T = T,
        cap_tightness = cap_tightness,
        buildtime1 = buildtime1, 
        solvetime1 = solvetime1, 
        objective1 = objective1,
        buildtime2 = buildtime2, 
        solvetime2 = solvetime2, 
        objective2 = objective2,

    ))
end
df3 = DataFrame(plotdata)
@show combine(groupby(df2, [:T, :S]), [:solvetime1, :solvetime2].=> mean)
@show combine(groupby(df2, [:cap_tightness]), [:solvetime1, :solvetime2].=> mean);

Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


...

Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


combine(groupby(df2, [:T, :S]), [:solvetime1, :solvetime2] .=> mean) = 9×4 DataFrame
 Row │ T      S      solvetime1_mean  solvetime2_mean
     │ Int64  Int64  Float64          Float64
─────┼────────────────────────────────────────────────
   1 │    10     30         0.404354         0.311926
   2 │    10    100         1.3831           6.05666
   3 │    10    200         3.19683         21.3332
   4 │    50     30         1.68796          0.286959
   5 │    50    100         7.6301           7.57418
   6 │    50    200        17.09            48.6835
   7 │   100     30         3.52868          0.265092
   8 │   100    100        17.0678           7.08576
   9 │   100    200        39.1949          61.6033


combine(groupby(df2, [:cap_tightness]), [:solvetime1, :solvetime2] .=> mean) = 2×3 DataFrame
 Row │ cap_tightness  solvetime1_mean  solvetime2_mean
     │ Int64          Float64          Float64
─────┼─────────────────────────────────────────────────
   1 │             0          10.8884          20.7382
   2 │             1           9.3747          13.3063


In [14]:
@show combine(groupby(df, [:cap_tightness]), [:solvetime1, :solvetime2].=> mean);
#@show combine(groupby(df2, [:cap_tightness]), [:solvetime1, :solvetime2].=> mean);
@show combine(groupby(df_l2500, [:cap_tightness]), [:solvetime1, :solvetime2].=> mean);

UndefVarError: UndefVarError: df not defined

In [11]:
groupedbar(
    repeat(lpad.(string.(L_range), 4), outer = 3), 
    vcat(df[!, :solvetime1], df[!, :solvetime2], df[!, :solvetime3]),
    group = repeat(["Model 1", "Model 2", "Model 3"], inner = 3),
    ylabel = "Solve time (s)",
    # yscale = :log10,
    title = "Solve times of both formulations with # of customers",
    legend = :topleft,
)

ArgumentError: ArgumentError: column name "solvetime3" not found in the data frame; existing most similar names are: "solvetime1" and "solvetime2"